###### Imports and Settings

In [1]:
import pandas as pd
import numpy as np
import requests
from functools import reduce
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)

# This notebook outlines the download and formatting process for the Zillow Home Value and Rental indices for counties and places in the GNRC operating region.  

Go to this page: https://www.zillow.com/research/data/  

+ Under "HOME VALUES", select Data Type "ZHVI All Homes (SFR, Condo/Co-op) Time Series, Smoothed, Seasonally Adjusted" and download this for Metro & US, State, County, and City. 
+ Under "RENTALS", select Data Type "ZORI (Smoothed, Seasonally Adjusted): All Homes Plus Multifamily Time Series" and download this for Metro & US, County, and City (State is not currently available).  
+ Under "INVENTORY", select Data Types: "For Sale Inventory (Smooth, All Homes, Monthly)", "New Listings (Smooth, All Homes, Monthly)", "Newly Pending Listings (Smooth, All Homes, Monthly)", and Mean Days to Pending (Smooth, All Homes, Monthly)" and download these for Metro & US (they are currently only available at this level)  
+ Under "LIST AND SALE PRICES" select Data Types: "Median List Price (Smooth, All Homes, Monthly)", "Median Sale Price (Smooth, All Homes, Monthly)", and "Percent of Homes Sold Above List (Smooth, All Homes, Monthly View)" and download these for Metro & US (they are currently only available at this level)  

### Save these csvs as they come in the Data Downloads folder of Parent Data Gathering

In [30]:
hv_metrous = pd.read_csv('../../Data Downloads/Metro_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')
hv_state = pd.read_csv('../../Data Downloads/State_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')
hv_county = pd.read_csv('../../Data Downloads/County_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')
hv_place = pd.read_csv('../../Data Downloads/City_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')
rental_metrous = pd.read_csv('../../Data Downloads/Metro_zori_sm_sa_month.csv')
rental_county = pd.read_csv('../../Data Downloads/County_zori_sm_sa_month.csv')
rental_place = pd.read_csv('../../Data Downloads/City_zori_sm_sa_month.csv')
inventory_metrous = pd.read_csv('../../Data Downloads/Metro_invt_fs_uc_sfrcondo_sm_month.csv')
newlistings_metrous = pd.read_csv('../../Data Downloads/Metro_new_listings_uc_sfrcondo_sm_month.csv')
newlypending_metrous = pd.read_csv('../../Data Downloads/Metro_new_pending_uc_sfrcondo_sm_month.csv')
meantopending_metrous = pd.read_csv('../../Data Downloads/Metro_mean_doz_pending_uc_sfrcondo_sm_month.csv')
medlisting_metrous = pd.read_csv('../../Data Downloads/Metro_mlp_uc_sfrcondo_sm_month.csv')
medsale_metrous = pd.read_csv('../../Data Downloads/Metro_median_sale_price_uc_sfrcondo_sm_month.csv')
pctabovelist_metrous = pd.read_csv('../../Data Downloads/Metro_pct_sold_above_list_uc_sfrcondo_sm_month.csv')

In [32]:
#filter down the metro and us files to include only the US and the Nashville and Clarksville MSAs
metrodfs = [hv_metrous,rental_metrous,inventory_metrous,newlistings_metrous,newlypending_metrous,meantopending_metrous,
            medlisting_metrous,medsale_metrous,pctabovelist_metrous]
for i in metrodfs:
    i = i = hv_metrous.loc[(i['RegionID'] == 102001)|(i['RegionID'] == 394902)|(i['RegionID'] == 394471)].reset_index(drop = True)
# hv_metrous = hv_metrous.loc[(hv_metrous['RegionID'] == 102001)|(hv_metrous['RegionID'] == 394902)|(hv_metrous['RegionID'] == 394471)].reset_index(drop = True)
# rental_metrous = rental_metrous.loc[(rental_metrous['RegionID'] == 102001)|(rental_metrous['RegionID'] == 394902)|
#                                     (rental_metrous['RegionID'] == 394471)].reset_index(drop = True)
# inventory_metrous = inventory_metrous.loc[(inventory_metrous['RegionID'] == 102001)|(inventory_metrous['RegionID'] == 394902)|
#                                           (inventory_metrous['RegionID'] == 394471)].reset_index(drop = True)
# newlistings_metrous = newlistings_metrous.loc[(newlistings_metrous['RegionID'] == 102001)|(newlistings_metrous['RegionID'] == 394902)|
#                                               (newlistings_metrous['RegionID'] == 394471)].reset_index(drop = True)
# newlypending_metrous = newlypending_metrous.loc[(newlypending_metrous['RegionID'] == 102001)|(newlypending_metrous['RegionID'] == 394902)|
#                                                 (newlypending_metrous['RegionID'] == 394471)].reset_index(drop = True)
# meantopending_metrous = meantopending_metrous.loc[(meantopending_metrous['RegionID'] == 102001)|(meantopending_metrous['RegionID'] == 394902)|
#                                                   (meantopending_metrous['RegionID'] == 394471)].reset_index(drop = True)
# medlisting_metrous = medlisting_metrous.loc[(medlisting_metrous['RegionID'] == 102001)|(medlisting_metrous['RegionID'] == 394902)|
#                                             (medlisting_metrous['RegionID'] == 394471)].reset_index(drop = True)
# medsale_metrous = medsale_metrous.loc[(medsale_metrous['RegionID'] == 102001)|(medsale_metrous['RegionID'] == 394902)|
#                                       (medsale_metrous['RegionID'] == 394471)].reset_index(drop = True)
# pctabovelist_metrous = pctabovelist_metrous.loc[(pctabovelist_metrous['RegionID'] == 102001)|(pctabovelist_metrous['RegionID'] == 394902)|(pctabovelist_metrous['RegionID'] == 394471)].reset_index(drop = True)

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [24]:
test.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,2000-06-30,2000-07-31,2000-08-31,2000-09-30,2000-10-31,2000-11-30,2000-12-31,2001-01-31,2001-02-28,2001-03-31,2001-04-30,2001-05-31,2001-06-30,2001-07-31,2001-08-31,2001-09-30,2001-10-31,2001-11-30,2001-12-31,2002-01-31,2002-02-28,2002-03-31,2002-04-30,2002-05-31,2002-06-30,2002-07-31,2002-08-31,2002-09-30,2002-10-31,2002-11-30,2002-12-31,2003-01-31,2003-02-28,2003-03-31,2003-04-30,2003-05-31,2003-06-30,2003-07-31,2003-08-31,2003-09-30,2003-10-31,2003-11-30,2003-12-31,2004-01-31,2004-02-29,2004-03-31,2004-04-30,2004-05-31,2004-06-30,2004-07-31,2004-08-31,2004-09-30,2004-10-31,2004-11-30,2004-12-31,2005-01-31,2005-02-28,2005-03-31,2005-04-30,2005-05-31,2005-06-30,2005-07-31,2005-08-31,2005-09-30,2005-10-31,2005-11-30,2005-12-31,2006-01-31,2006-02-28,2006-03-31,2006-04-30,2006-05-31,2006-06-30,2006-07-31,2006-08-31,2006-09-30,2006-10-31,2006-11-30,2006-12-31,2007-01-31,2007-02-28,2007-03-31,2007-04-30,2007-05-31,2007-06-30,2007-07-31,2007-08-31,2007-09-30,2007-10-31,2007-11-30,2007-12-31,2008-01-31,2008-02-29,2008-03-31,2008-04-30,2008-05-31,2008-06-30,2008-07-31,2008-08-31,2008-09-30,2008-10-31,2008-11-30,2008-12-31,2009-01-31,2009-02-28,2009-03-31,2009-04-30,2009-05-31,2009-06-30,2009-07-31,2009-08-31,2009-09-30,2009-10-31,2009-11-30,2009-12-31,2010-01-31,2010-02-28,2010-03-31,2010-04-30,2010-05-31,2010-06-30,2010-07-31,2010-08-31,2010-09-30,2010-10-31,2010-11-30,2010-12-31,2011-01-31,2011-02-28,2011-03-31,2011-04-30,2011-05-31,2011-06-30,2011-07-31,2011-08-31,2011-09-30,2011-10-31,2011-11-30,2011-12-31,2012-01-31,2012-02-29,2012-03-31,2012-04-30,2012-05-31,2012-06-30,2012-07-31,2012-08-31,2012-09-30,2012-10-31,2012-11-30,2012-12-31,2013-01-31,2013-02-28,2013-03-31,2013-04-30,2013-05-31,2013-06-30,2013-07-31,2013-08-31,2013-09-30,2013-10-31,2013-11-30,2013-12-31,2014-01-31,2014-02-28,2014-03-31,2014-04-30,2014-05-31,2014-06-30,2014-07-31,2014-08-31,2014-09-30,2014-10-31,2014-11-30,2014-12-31,2015-01-31,2015-02-28,2015-03-31,2015-04-30,2015-05-31,2015-06-30,2015-07-31,2015-08-31,2015-09-30,2015-10-31,2015-11-30,2015-12-31,2016-01-31,2016-02-29,2016-03-31,2016-04-30,2016-05-31,2016-06-30,2016-07-31,2016-08-31,2016-09-30,2016-10-31,2016-11-30,2016-12-31,2017-01-31,2017-02-28,2017-03-31,2017-04-30,2017-05-31,2017-06-30,2017-07-31,2017-08-31,2017-09-30,2017-10-31,2017-11-30,2017-12-31,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30
0,102001,0,United States,country,NaN,128394.0,128741.0,129106.0,129850.0,130601.0,131349.0,132112.0,132925.0,133757.0,134592.0,135436.0,136300.0,137107.0,137848.0,138602.0,139407.0,140236.0,141049.0,141869.0,142717.0,143556.0,144401.0,145174.0,145913.0,146587.0,147290.0,148032.0,148817.0,149654.0,150534.0,151489.0,152486.0,153472.0,154463.0,155397.0,156329.0,157179.0,158045.0,158934.0,159894.0,160922.0,161947.0,163016.0,164145.0,165339.0,166532.0,167610.0,168639.0,169664.0,170841.0,172155.0,173592.0,175181.0,176872.0,178673.0,180443.0,182169.0,183810.0,185345.0,186905.0,188437.0,190045.0,191715.0,193632.0,195629.0,197709.0,199766.0,201836.0,203837.0,205697.0,207359.0,208849.0,210108.0,211326.0,212599.0,213977.0,215291.0,216396.0,217203.0,217813.0,218153.0,218398.0,218475.0,218520.0,218502.0,218521.0,218529.0,218547.0,218387.0,218033.0,217419.0,216769.0,215978.0,215154.0,214118.0,213070.0,211820.0,210467.0,208952.0,207369.0,205773.0,

In [26]:
hv_metrous.head(200)

,RegionID,SizeRank,RegionName,RegionType,StateName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,2000-06-30,2000-07-31,2000-08-31,2000-09-30,2000-10-31,2000-11-30,2000-12-31,2001-01-31,2001-02-28,2001-03-31,2001-04-30,2001-05-31,2001-06-30,2001-07-31,2001-08-31,2001-09-30,2001-10-31,2001-11-30,2001-12-31,2002-01-31,2002-02-28,2002-03-31,2002-04-30,2002-05-31,2002-06-30,2002-07-31,2002-08-31,2002-09-30,2002-10-31,2002-11-30,2002-12-31,2003-01-31,2003-02-28,2003-03-31,2003-04-30,2003-05-31,2003-06-30,2003-07-31,2003-08-31,2003-09-30,2003-10-31,2003-11-30,2003-12-31,2004-01-31,2004-02-29,2004-03-31,2004-04-30,2004-05-31,2004-06-30,2004-07-31,2004-08-31,2004-09-30,2004-10-31,2004-11-30,2004-12-31,2005-01-31,2005-02-28,2005-03-31,2005-04-30,2005-05-31,2005-06-30,2005-07-31,2005-08-31,2005-09-30,2005-10-31,2005-11-30,2005-12-31,2006-01-31,2006-02-28,2006-03-31,2006-04-30,2006-05-31,2006-06-30,2006-07-31,2006-08-31,2006-09-30,2006-10-31,2006-11-30,2006-12-31,2007-01-31,2007-02-28,2007-03-31,2007-04-30,2007-05-31,2007-06-30,2007-07-31,2007-08-31,2007-09-30,2007-10-31,2007-11-30,2007-12-31,2008-01-31,2008-02-29,2008-03-31,2008-04-30,2008-05-31,2008-06-30,2008-07-31,2008-08-31,2008-09-30,2008-10-31,2008-11-30,2008-12-31,2009-01-31,2009-02-28,2009-03-31,2009-04-30,2009-05-31,2009-06-30,2009-07-31,2009-08-31,2009-09-30,2009-10-31,2009-11-30,2009-12-31,2010-01-31,2010-02-28,2010-03-31,2010-04-30,2010-05-31,2010-06-30,2010-07-31,2010-08-31,2010-09-30,2010-10-31,2010-11-30,2010-12-31,2011-01-31,2011-02-28,2011-03-31,2011-04-30,2011-05-31,2011-06-30,2011-07-31,2011-08-31,2011-09-30,2011-10-31,2011-11-30,2011-12-31,2012-01-31,2012-02-29,2012-03-31,2012-04-30,2012-05-31,2012-06-30,2012-07-31,2012-08-31,2012-09-30,2012-10-31,2012-11-30,2012-12-31,2013-01-31,2013-02-28,2013-03-31,2013-04-30,2013-05-31,2013-06-30,2013-07-31,2013-08-31,2013-09-30,2013-10-31,2013-11-30,2013-12-31,2014-01-31,2014-02-28,2014-03-31,2014-04-30,2014-05-31,2014-06-30,2014-07-31,2014-08-31,2014-09-30,2014-10-31,2014-11-30,2014-12-31,2015-01-31,2015-02-28,2015-03-31,2015-04-30,2015-05-31,2015-06-30,2015-07-31,2015-08-31,2015-09-30,2015-10-31,2015-11-30,2015-12-31,2016-01-31,2016-02-29,2016-03-31,2016-04-30,2016-05-31,2016-06-30,2016-07-31,2016-08-31,2016-09-30,2016-10-31,2016-11-30,2016-12-31,2017-01-31,2017-02-28,2017-03-31,2017-04-30,2017-05-31,2017-06-30,2017-07-31,2017-08-31,2017-09-30,2017-10-31,2017-11-30,2017-12-31,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30
0,102001,0,United States,country,NaN,128394.0,128741.0,129106.0,129850.0,130601.0,131349.0,132112.0,132925.0,133757.0,134592.0,135436.0,136300.0,137107.0,137848.0,138602.0,139407.0,140236.0,141049.0,141869.0,142717.0,143556.0,144401.0,145174.0,145913.0,146587.0,147290.0,148032.0,148817.0,149654.0,150534.0,151489.0,152486.0,153472.0,154463.0,155397.0,156329.0,157179.0,158045.0,158934.0,159894.0,160922.0,161947.0,163016.0,164145.0,165339.0,166532.0,167610.0,168639.0,169664.0,170841.0,172155.0,173592.0,175181.0,176872.0,178673.0,180443.0,182169.0,183810.0,185345.0,186905.0,188437.0,190045.0,191715.0,193632.0,195629.0,197709.0,199766.0,201836.0,203837.0,205697.0,207359.0,208849.0,210108.0,211326.0,212599.0,213977.0,215291.0,216396.0,217203.0,217813.0,218153.0,218398.0,218475.0,218520.0,218502.0,218521.0,218529.0,218547.0,218387.0,218033.0,217419.0,216769.0,215978.0,215154.0,214118.0,213070.0,211820.0,210467.0,208952.0,207369.0,205773.0,